# Obtencion de datos


El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD.
El sistema/base de datos es TAS (SISTEMA DE MERCADO DE DINERO Y CAMBIOS) el cual registra y controla y administra las operaciones que realizan el area de Mercado de Dinero con clientes e intermediarios, refente a comprar y ventas de los diferentes instrumentos del mercado , asi como el registro, control y administracion las operaciones.

Para poder describir la informacion de la base de datos, se debe conocer ¿Quien es el cliente?

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |                                                  |
| nombre cliente o razón social           |          "U##NOMBRE", NOMBRE, NOMLARGO    |
| número cliente                          |          CUENTA                               |
| estatus                                 |         FBAJA IS NULL                            |
| número contrato                         |                                     |
| género                                  |                      N/A                            |
| fecha_nacimiento o de constitución      |                                                  |
| entidad federativa nacimiento           |                                                  |
| país de nacimiento                      |    if INACIONALIDAD == 1 then "MEXICO" else "No se cuenta con la informacion"                                                |
| nacionalidad                            |   INACIONALIDAD  esta columna proviene de un catálogo de la tabla TAS.FNACIONALIDAD                             |
| profesión                               |                    IACTIV_ECONO                              |
| calle                                   |                                 |
| número_exterior                         |                                                  |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |                                                  |
| ciudad_población                        |                                                  |
| entidad_federativa                      |                                    |
| código postal                           |                                                  |
| país                                    |                                |
| teléfono                                |    TELEFONO1, TELEFONO2                                |
| correo_electrónico                      |                    EMAIL                              |
| rfc                                     |  RFC   |
| curp                                    |  CURP                                                |
| fiel                                    |                                                  |
| representante legal                     |   Esta informacion esta en la tabla TAS.FAPODERADO se  debe hacer un join                                                |
| tipo de persona                         |  TIPINVE esta columa proviene de un catálogo de la tabla TAS.TIPINVER  |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     |                   |
| cambios en perfil transaccional         |                                                  |


- Esta informacion no se puede obtener de la base de datos, ya que no se cuenta con la descripcion de requerimiento

    - clasificacion por grado de riesgo	
    - razon
    - cambios en perfil transaccional 

In [112]:
query = """
WITH CLIENTES_ACTIVOS AS(
	SELECT * 
    FROM TAS.CLIENTE cl
    WHERE cl.FBAJA IS NULL
), 
CLIENTES_CON_NA AS(
	SELECT CA.*, FN.NNACIONALIDAD NACIONALIDAD
	FROM CLIENTES_ACTIVOS CA
	INNER JOIN TAS.FNACIONALIDAD FN ON CA.INACIONALIDAD = FN.INACIONALIDAD
), 
CLIENTES_CON_TRANS_2024 AS (
	SELECT CCN.*, ft.FOPER, ft.FLIQ, ft.IORDEN, ft.ITINSTR ,ft.IINSTR, 
			ft."U##IEMISION", ft.ISERIE, ft.MBRUTO, ft.MCOSTO, ft.NTITULOS
	FROM CLIENTES_CON_NA CCN
	LEFT JOIN TAS.FTRANS ft ON CCN.CUENTA = ft.ICONTRATO
	WHERE FOPER > TO_DATE('2024-01-01', 'YYYY-MM-DD')
), 
CLIENTES_CON_TIP_INV AS (
	SELECT CCT.*, tp.DESCRIP TIPO_DE_PERSONA
	FROM CLIENTES_CON_TRANS_2024 CCT
	LEFT JOIN TAS.TIPINVER tp ON CCT.TIPINVE = tp.TIPINVE
) SELECT 
	'' NEGOCIO, 
	'' NOMBRE_O_RAZON_SOCIAL,
	"U##NOMBRE" , NOMBRE, NOMLARGO,
	CUENTA, 
	'VIGENTE' ESTATUS, 
	'' NUMERO_CONTRATO,
	'' GENERO,  
	'' FECHA_NAC_O_CONST, 
	'' ENTIDAD_FEDERATIVA_NACIMIENTO, 
	'' PAIS_DE_NACIMIENTO, 
	NACIONALIDAD,
	IACTIV_ECONO PROFESION,
	'' CALLE,
	'' NUMERO_EXTERIOR, 
	'' NUMERO_INTERIOR, 
	'' COLONIA_URBANIZACION, 
	'' DELEGACION_MUNICIPIO, 
	'' CIUDAD_POBLACION,  
	'' ENTIDAD_FEDERATIVA, 
	'' CODIGO_POSTAL, 
	IREGIONAL PAIS,
	'' TELEFONO, 
	TELEFONO1 , TELEFONO2,
	EMAIL CORREO_ELECTRONICO, 
	RFC RFC, 
	'' CURP, 
	'' FIEL,
	'' REPRESENTANTE_LEGAL,
	TIPO_DE_PERSONA TIPO_PERSONA, 
	'' PRODUCTO_CONTRATADO, 
	TIPINVE, FOPER, FLIQ, IORDEN, ITINSTR , IINSTR, "U##IEMISION", ISERIE, MBRUTO, MCOSTO, NTITULOS, 
	'TAS' SISTEMA_ORIGEN
FROM CLIENTES_CON_TIP_INV
"""

query2 = "SELECT CUENTA, APE_PAT || ' ' || APE_MAT || ' ' || NOMBRE AS REPRESENTANTE_LEGAL FROM TAS.FAPODERADO"

In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from datetime import datetime
import cx_Oracle
import pandas as pd
import os

In [109]:
def get_data_tas(database,path_exit, query, query2):
    conn = cx_Oracle.connect(user=database['USER'], password=database["PSSWD"], dsn=database["DSN"])
    cursor = conn.cursor()
    
    cursor.execute(query)
    columnas_clientes = [columna[0] for columna in cursor.description]
    tas_clientes = pd.DataFrame(cursor.fetchall(), columns=columnas_clientes)
    
    cursor.execute(query2)
    columnas_fapoderado = [columna[0] for columna in cursor.description]
    fapoderado =   pd.DataFrame(cursor.fetchall(), columns=columnas_fapoderado)

    representa_legal = fapoderado.groupby('CUENTA')['REPRESENTANTE_LEGAL'].apply(lambda x: ', '.join(x)).reset_index()
    
    resultado = pd.merge(tas_clientes, representa_legal, on="CUENTA", how="left")
    resultado.to_csv(os.path.join(path_exit,"Transformacion_clientes_TAS.dat"),index=False)
    
    cursor.close()
    conn.close()
    

In [110]:
path_exit = "/home/ale1726/proyects/datalake/clientes/data"


In [113]:
get_data_tas(db_tas, path_exit, query, query2)

In [2]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Transformacion_clientes_TAS.dat")

In [ ]:
df

In [103]:
df_fitlred = df[df["CUENTA"] == 76000]

In [104]:
df_cts_pld = pd.read_csv("/home/ale1726/proyects/dash/dash-anlytc_data/data/Acumulado_clientes.csv")

In [106]:
df_cts_pld.value_counts("producto contratado")

producto contratado
Contrato de Depósito Bancario de Títulos en Administración y de Comisión Mercantil No Discrecional                                                                          1254
Contrato Marco Para Operaciones de Compraventa de Valores y Reporto                                                                                                          697
FIDEICOMISO                                                                                                                                                                  539
Línea de Crédito                                                                                                                                                              95
Programa Primer Piso                                                                                                                                                          57
Contrato de Depósito y Garantía Prendaria                                                      

In [105]:
df_cts_pld.columns

Index(['negocio', 'nombre cliente o razon social', 'numero cliente', 'estatus',
       'numero contrato', 'genero', 'fecha_nacimiento o de constitucion',
       'entidad federativa nacimiento', 'pais de nacimiento', 'nacionalidad',
       'profesion', 'calle', 'numero_exterior', 'numero_interior',
       'colonia_urbanizacion', 'delegacion_municipio', 'ciudad_poblacion',
       'entidad_federativa', 'codigopostal', 'pais', 'telefono',
       'correo_electronico', 'rfc', 'curp', 'fiel', 'representante legal',
       'tipo de persona', 'clasificacion por grado de riesgo',
       'pep (persona políticamente expuesta)', 'razon', 'producto contratado',
       'cambios en perfil transaccional'],
      dtype='object')

In [ ]:
df_fitlred

In [ ]:
df.head()

,CUENTA,U##NOMBRE,NOMBRE,NOMLARGO,ESTATUS,NUMERO DE CONTRATO,INACIONALIDAD,IACTIV_ECONO,TELEFONO1,TELEFONO2,...,FLIQ,IORDEN,ITINSTR,IINSTR,U##IEMISION,ISERIE,MBRUTO,MCOSTO,NTITULOS,REPRESENTANTE_LEGAL
0,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136612,GUBERR,BPA182,BPA182,IS300328,5.000000e+09,5.189662e+09,51322766,NaN
1,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136611,GUBERR,BPA182,BPA182,IS300328,5.189662e+09,5.189662e+09,51322766,NaN
2,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136610,GUBERR,BONDESF,BONDESF,LF251023,5.000000e+09,5.082295e+09,50497249,NaN
3,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136609,GUBERR,BONDESF,BONDESF,LF251023,5.082295e+09,5.082295e+09,50497249,NaN
4,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-18 00:00:00,135801,GUBERR,BPAG91,BPAG91,IQ270506,5.000000e+09,5.080935e+09,49963230,NaN


In [ ]:
r1 = len(df[df["U##NOMBRE"].isna()])
r2 = len(df[df["NOMBRE"].isna()])
r3 = len(df[df["NOMBRE"].isna()])
print(r1,r2,r3)

0 0 0


In [83]:
df_filtred = df.drop_duplicates("NOMLARGO")
df_filtred = df_filtred[["U##NOMBRE", "NOMBRE", "NOMLARGO"]]
df_filtred["VERIFICACION"] = df_filtred.apply(lambda row: row["U##NOMBRE"] == row["NOMBRE"] == row["NOMLARGO"], axis=1)

In [85]:
df_filtred.value_counts("VERIFICACION")

VERIFICACION
False    520
True       5
Name: count, dtype: int64

In [88]:
df_ver = df_filtred[df_filtred["VERIFICACION"] == False]

In [90]:
df_ver["NOMLARGO"] = df_ver["NOMLARGO"].apply(lambda x: x.decode('ascii') if isinstance(x, bytes) else x)

/tmp/ipykernel_10337/950514676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ver["NOMLARGO"] = df_ver["NOMLARGO"].apply(lambda x: x.decode('ascii') if isinstance(x, bytes) else x)


In [ ]:
texto_mal_codificado = "FONDO BBVA M�XICO DEUDA EMPRESAS, S.A. DE C.V. F.I.I.D."

# Decodificar el texto directamente desde utf-8
try:
    texto_decodificado = texto_mal_codificado.encode('latin1').decode('utf-8')
except UnicodeEncodeError:
    print("Error al codificar el texto con latin1.")

Error al codificar el texto con latin1.


In [ ]:
df_ver["NOMLARGO"] = df_ver["NOMLARGO"].apply(lambda x: x.upper())

In [ ]:
df_ver

In [ ]:
def nombre(df):
    